# Cryptocurrency trading bot.

In [1]:
##%pip install python-binance==0.7.5
#%pip install python-binance

In [2]:
!python bootstrap.py

100%|████████████████████████████████████| 1431/1431 [19:34<00:00,  1.22 pair/s]
1431 named pair [02:37,  9.07 named pair/s]
100%|████████████████████████████████████| 1431/1431 [08:59<00:00,  2.65 pair/s]
1431 named pair [00:13, 109.83 named pair/s]
100%|████████████████████████| 1431/1431 [00:05<00:00, 261.33 pair conversion/s]
1431 named pair [01:29, 15.96 named pair/s]
100%|████████████████████████| 1431/1431 [00:07<00:00, 185.75 pair conversion/s]
1431 named pair [01:31, 15.72 named pair/s]
100%|██████████████████████████████████| 1431/1431 [00:00<00:00, 5093.44 pair/s]
1431 named pair [01:45, 13.56 named pair/s]
100%|██████████████████████████████████| 1431/1431 [00:00<00:00, 3553.35 pair/s]
1431 named pair [01:39, 14.43 named pair/s]
  0%|                                                  | 0/3 [00:00<?, ? pair/s]
Traceback (most recent call last):
  File "bootstrap.py", line 43, in <module>
    main()
  File "bootstrap.py", line 37, in main
    pairs = bootstrap_loggers(client=c